# Clickbait Spoiler Generation using BERT

In [1]:
# This is necessary to fix the imports
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../src')))

import utils.bert
from utils.bert import MODEL_CHECKPOINT, TOKENIZER
from transformers import TFAutoModelForQuestionAnswering, DefaultDataCollator, create_optimizer, pipeline

import tensorflow as tf

from datasets import load_dataset

import evaluate

/home/rafael/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-29 21:24:35.714430: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 21:24:36.319960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package wordnet to /home/rafael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/rafael/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rafael/nlt

In [41]:
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

[nltk_data] Downloading package wordnet to /home/rafael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/rafael/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rafael/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
BATCH_SIZE = 3
N_EPOCHS = 5
LEARNING_RATE = 2e-5
# SAVE_CHECKPOINT_PATH = "../src/models/bert/clickbait"

SAVE_CHECKPOINT_PATH = "../src/models/bert/clickbait_deberta"

In [6]:
model = TFAutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT, from_pt=True)

2023-05-29 21:26:57.648039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 21:26:57.678972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 21:26:57.679044: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 21:26:57.680891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 21:26:57.680953: I tensorflow/compile

In [7]:
raw_datasets = load_dataset("../data/parsed/bert/clickbait_data.py", data_files={
    "train": "train.jsonl", 
    "test": "test.jsonl", 
    "validation": "validation.jsonl"
})

Found cached dataset clickbait_data (/home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 942.68it/s]


In [8]:
train_dataset = raw_datasets["train"].filter(lambda x: x["type"] == "phrase").map(
    utils.bert.preprocess_training,
    batched=True,
    remove_columns=raw_datasets["train"].filter(lambda x: x["type"] == "phrase").column_names,
)
len(raw_datasets["train"].filter(lambda x: x["type"] == "phrase")), len(train_dataset)

Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-2ad544e5be377783.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-2ad544e5be377783.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-44c7fa42af0a0960.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-2ad544e5be377783.arrow


(1367, 3192)

In [9]:
test_dataset = raw_datasets["test"].filter(lambda x: x["type"] == "phrase").map(
    utils.bert.preprocess_validation,
    batched=True,
    remove_columns=raw_datasets["test"].filter(lambda x: x["type"] == "phrase").column_names,
)
len(raw_datasets["test"].filter(lambda x: x["type"] == "phrase")), len(test_dataset)

Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-0bd9b95764e6d05e.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-0bd9b95764e6d05e.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-08fb061e3ed8a5ad.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-0bd9b95764e6d05e.arrow


(268, 585)

In [10]:
validation_dataset = raw_datasets["validation"].filter(lambda x: x["type"] == "phrase").map(
    utils.bert.preprocess_validation,
    batched=True,
    remove_columns=raw_datasets["validation"].filter(lambda x: x["type"] == "phrase").column_names,
)
len(raw_datasets["validation"].filter(lambda x: x["type"] == "phrase")), len(validation_dataset)

Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-dcec741138fad6a0.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-dcec741138fad6a0.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-f8776e9313f1e434.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-dcec741138fad6a0.arrow


(67, 182)

In [11]:
data_collator = DefaultDataCollator(return_tensors="tf")

In [12]:
tf_train_dataset = model.prepare_tf_dataset(
    train_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [13]:
tf_validation_dataset = model.prepare_tf_dataset(
    validation_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [14]:
num_train_steps = len(tf_train_dataset) * N_EPOCHS
optimizer, schedule = create_optimizer(
    init_lr=LEARNING_RATE,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [15]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir="../src/models/bert/logs"),
]

In [16]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [17]:
print(tf.config.list_physical_devices("GPU"))
history = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    callbacks=callbacks,
    epochs=N_EPOCHS,
    verbose=1,
)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Epoch 1/5


2023-05-29 21:28:53.205843: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [3192]
	 [[{{node Placeholder/_0}}]]
2023-05-29 21:28:53.206022: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [3192]
	 [[{{node Placeholder/_0}}]]


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
1064/1064 [==============================] - ETA: 0s - loss: 3.8072

2023-05-29 21:34:36.735125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [182]
	 [[{{node Placeholder/_0}}]]
2023-05-29 21:34:36.735315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [182]
	 [[{{node Placeholder/_0}}]]


1064/1064 [==============================] - 345s 311ms/step - loss: 3.8072 - val_loss: 0.0000e+00
Epoch 2/5
1064/1064 [==============================] - 326s 306ms/step - loss: 3.2896 - val_loss: 0.0000e+00
Epoch 3/5
1064/1064 [==============================] - 323s 303ms/step - loss: 3.2647 - val_loss: 0.0000e+00
Epoch 4/5
1064/1064 [==============================] - 325s 305ms/step - loss: 3.2586 - val_loss: 0.0000e+00
Epoch 5/5
1064/1064 [==============================] - 325s 306ms/step - loss: 3.2580 - val_loss: 0.0000e+00


In [18]:
model.save_pretrained(SAVE_CHECKPOINT_PATH)

In [21]:
model = TFAutoModelForQuestionAnswering.from_pretrained(SAVE_CHECKPOINT_PATH)

Some layers from the model checkpoint at ../src/models/bert/clickbait were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at ../src/models/bert/clickbait and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
tf_test_dataset = model.prepare_tf_dataset(
    test_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [20]:
predictions = model.predict(tf_test_dataset)

2023-05-29 22:03:59.929326: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [585]
	 [[{{node Placeholder/_0}}]]
2023-05-29 22:03:59.929506: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [585]
	 [[{{node Placeholder/_0}}]]


195/195 [==============================] - 16s 71ms/step


In [21]:
utils.bert.compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_dataset,
    raw_datasets["validation"],
)

100%|██████████████████████████████████████| 198/198 [00:00<00:00, 14753.20it/s]


{'squad': {'exact_match': 0.5050505050505051, 'f1': 0.0},
 'meteor': {'meteor': 0.0},
 'bertscore': {'precision': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,


In [22]:
question_answerer = pipeline("question-answering", model=MODEL_CHECKPOINT, tokenizer=TOKENIZER)

In [67]:
idx = 14

context = raw_datasets["validation"].filter(lambda x: x["type"] == "phrase")[idx]['context']
question = raw_datasets["validation"].filter(lambda x: x["type"] == "phrase")[idx]['question']
expected = raw_datasets["validation"].filter(lambda x: x["type"] == "phrase")[idx]['answers']['text'][0]

Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-dcec741138fad6a0.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-dcec741138fad6a0.arrow
Loading cached processed dataset at /home/rafael/.cache/huggingface/datasets/clickbait_data/default-cf38bafdeefc1216/0.0.0/2132235c1c29143999b3a1b191327cbddac13587917dfde07b49cb535c8668f7/cache-dcec741138fad6a0.arrow


In [68]:
context

'When Donald Trump declared his candidacy for president a year ago, Nate Silver’s FiveThirtyEight political blog said he had "a better chance of cameoing in another ‘Home Alone’ movie with Macaulay Culkin—or playing in the NBA Finals—than winning the Republican nomination." With a net favorability rating of -32 percent according to FiveThirtyEight and "exactly zero" chance of winning, according to The Atlantic, how has Trump seemingly emerged victorious from a pool of 17 Republican hopefuls? To try to answer that question would be to travel down a rabbit hole of speculation that has swallowed so many. One thing is sure, though: While Donald Trump has thus far appeared to accomplish the impossible, even he has limits. The reality is that Trump—like policymakers and politicians past, present and future—is operating within the confines of the Overton Window of Political Possibility. The Overton Window: When You Think About It, It’s So True... This theory, named for the late Joseph P. Over

In [69]:
prediction = question_answerer(question=question, context=context)
prediction

{'score': 2.177469696107437e-06,
 'start': 3433,
 'end': 3495,
 'answer': ' Window—appeal to the hearts and minds of your fellow citizens'}

In [70]:
meteor_results = meteor.compute(predictions=[prediction["answer"]], references=[expected])
bleu_results = bleu.compute(predictions=[prediction["answer"]], references=[expected])
bertscore_results = bertscore.compute(predictions=[prediction["answer"]], references=[expected], lang="en")

In [71]:
print(f"{question};\n\nExpected Spoiler: {expected};\n\nSpoiler Predicted: {prediction['answer']};\n\n###\n\n")
print(f"Meteor: {meteor_results['meteor']}\nBLEU-4: {bleu_results['bleu']}\nBERTscore Mean F1: {sum(bertscore_results['f1'])/len(bertscore_results['f1'])}")

How One Theory You Never Heard of Explains Everything Donald Trump Does;

Expected Spoiler: The Overton Window;

Spoiler Predicted:  Window—appeal to the hearts and minds of your fellow citizens;

###


Meteor: 0.13513513513513514
BLEU-4: 0.0
BERTscore Mean F1: 0.800987958908081
